### 导入数据并观察

In [ ]:
# 忽视警告，这个库是内置的，不需要安装
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas  as pd

data = pd.read_csv('data.csv')
# # 随机打乱数据集，这一步的操作是可选的，类似于把练习题不断更改顺序，防止电脑学习到固定的顺序，这一步是可选的
# random_seed = 77
# data = data.sample(frac=1, random_state=random_seed)
data.head()

In [ ]:
# 从这里可以看出没有空缺值，所以不用处理缺失值
data.info()

In [ ]:
# 删除无用特征 customerID
data.drop('customerID', axis=1, inplace=True)
data.head()
data.info()

In [ ]:
data.shape

In [ ]:
# 删除特征后，我们对特征进行编码，这里我们对离散特征进行独特编码，连续特征做标准化处理
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder # 用于特征标准化、独热编码和数值编码
from sklearn.model_selection import train_test_split # 用于将数据集划分为训练集和测试集

# 分离特征与标签
X = data.drop('Churn', axis=1)
y = data['Churn']

# 删除高基数字段
X = X.drop(columns=["TotalCharges"])

X

## K-means 引入新的特征

### 使用肘部法则判断K值的选取

In [ ]:
import numpy as np  # 导入 NumPy 库，用于数值计算
from sklearn.cluster import KMeans  # 从 scikit-learn 库中导入 KMeans 聚类算法
from sklearn.manifold import TSNE  # 从 scikit-learn 库中导入 TSNE，用于降维
from sklearn.decomposition import PCA #导入pca
from scipy.spatial.distance import cdist  # 从 SciPy 库中导入 cdist 函数，用于计算距离
import matplotlib.pyplot as plt  # 导入 Matplotlib 库，用于数据可视化

# 设置 Matplotlib 字体以避免字体缺失的警告
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用黑体显示中文
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号

# 使用肘部法则确定最佳的 K 值
K = range(1, 10)  # 选择 K 的范围
mean_distortions = []  # 存储每个 K 值对应的平均畸变程度

for k in K:  # 遍历每个 K 值
    k_means = KMeans(n_clusters=k)  # 初始化 KMeans 模型，设置聚类数为 k
    k_means.fit(X)  # 训练 KMeans 模型
    centers = k_means.cluster_centers_  # 获取聚类中心
    error = sum(np.min(cdist(X, centers, 'euclidean'), axis=1))  # 计算每个点到其最近聚类中心的距离，并求和
    mean_distortions.append(error)  # 将误差添加到列表中

# 绘制肘部法则图
plt.plot(K, mean_distortions, 'bx-')  # 绘制 K 值与平均畸变程度的关系图，使用蓝色的 'x' 标记点，并用线连接
plt.xlabel('k')  # 设置 x 轴标签为 'k'
plt.ylabel('平均畸变程度')  # 设置 y 轴标签为 '平均畸变程度'
plt.title('用肘部法则来确定最佳的 K 值')  # 设置图表标题为 '用肘部法则来确定最佳的 K 值'
# plt.savefig("iris1.png", bbox_inches='tight')  # 保存图表为 'iris1.png' 文件，bbox_inches='tight' 表示紧凑布局
plt.show()  # 显示图表

### 使用K-means聚类分析引入新的特征，包括所属类别，距离质心的距离

In [ ]:
# 这里的最佳K值从上面获得，找下降幅度最大的点，这里是3
from sklearn.cluster import KMeans  # 从 scikit-learn 库中导入 KMeans 聚类算法
from scipy.spatial.distance import cdist  # 从 SciPy 库中导入 cdist 函数，用于计算距离

# 训练模型
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
# 读取聚类标签
cluster_labels = kmeans.labels_
# 计算距离质心距离
distances_to_centroids = kmeans.transform(X)
# 获取每个样本到其所属质心的距离
distance_to_own_centroid = distances_to_centroids[range(len(X)), kmeans.labels_]

# 将特征添加至数据集中
X['Cluster'] = cluster_labels
X['Distance_to_Centroid'] = distance_to_own_centroid

# 打印看看
X.head()

## 数据预处理

In [ ]:
# 可以看出标签有点不平衡，所以后面我们需要进行过采样
y.value_counts()

### 离散特征和连续特征编码

In [ ]:
# 如果需要填补缺失值，使用众数
# for column in X.columns:
#     if X[column].dtype in ['float64', 'int64']:
#         mode = X[column].mode()[0]
#         X[column].fillna(mode, inplace=True)

# 分离离散特征和连续特征  这里我们把 int 和 float 都看作连续特征， 字符串看作离散特征
continuous_features = X.select_dtypes(include=['float64', 'int64']).columns
discrete_features = X.select_dtypes(include=['object']).columns


# 对连续特征进行标准化处理
scaler = StandardScaler()
X_continuous = scaler.fit_transform(X[continuous_features])
# 对离散特征进行独热编码  // 这里可以考虑使用标签编码减少特征的数量
encoder = OneHotEncoder()
X_discrete = encoder.fit_transform(X[discrete_features])
# 合并连续特征和离散特征
X = pd.concat([pd.DataFrame(X_continuous), pd.DataFrame(X_discrete.toarray())], axis=1)

# 对标签进行数值编码
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# 使用 train_test_split 函数按照 8:2 的比例划分数据集
# test_size=0.2 表示 20%的数据用作测试集，即验证集。
# random_state 是一个随机数种子，确保每次划分的结果相同，便于复现结果。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 打印看看数据
# X_train.info()
X_test.info()

### SMOTE过采样

In [ ]:
# 对训练集数据进行过采样， 这里我们使用SMOTE插值法产生样本 不能对考试即测试集进行过采样

from imblearn.over_sampling import SMOTE # 导入 SMOTE 方法

# 使用 SMOTE 进行过采样,并赋值给自己
smote = SMOTE(random_state=42)  # 随机数种子我们还是设置为 42
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
y_train

## 模型训练

---
至此为止，数据初步处理已经完成，接下来训练模型

In [ ]:
# 这里我们的基模型选择使用逻辑回归、随机森林、XGBoost、LightGBM 四种模型
# 首先不调参，看看效果
# 导入所需的库
import pandas as pd  # 用于数据处理和分析
import numpy as np  # 用于数值计算
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder  # 用于数据预处理
from sklearn.model_selection import train_test_split, GridSearchCV  # 用于数据集划分和超参数调优
from sklearn.linear_model import LinearRegression  # 线性回归模型
from sklearn.linear_model import LogisticRegression  # 逻辑回归模型
from sklearn.svm import SVC  # 支持向量机分类模型
from sklearn.naive_bayes import GaussianNB  # 高斯朴素贝叶斯模型
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类模型
import xgboost as xgb  # XGBoost模型
import lightgbm as lgb  # LightGBM模型
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report  # 用于模型评估
import warnings
# 忽略所有警告
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt # 绘图

### 不调参情况

#### 逻辑回归

In [ ]:
# 逻辑回归
print('逻辑回归')

lr = LogisticRegression()  # 实例化逻辑回归模型
lr.fit(X_train, y_train)  # 训练模型
y_pred_lr = lr.predict(X_test)  # 预测

# y_pred_lr
# 输出
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))
print("Classification Report:\n", classification_report(y_test, y_pred_lr))
print("AUC: ", roc_auc_score(y_test, y_pred_lr))

#### 随机森林

In [ ]:
# 随机森林
print('随机森林')

# 定义随机森林模型（使用默认参数）
rf = RandomForestClassifier(random_state=42)

# 训练模型
rf.fit(X_train, y_train)

# 在测试集上进行预测
y_pred_proba = rf.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print(" AUC: ", roc_auc_score(y_test, y_pred_proba))

#### SVM

In [ ]:
# SVM
print('SVM')

# 定义 SVM 模型（使用默认参数）
svm = SVC(probability=True, random_state=42)  # 启用概率估计

# 训练模型
svm.fit(X_train, y_train)

# 在测试集上进行预测
y_pred_proba = svm.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))

#### XGBoost

In [ ]:
# XGBoost
print('XGBoost')

# 定义 XGBoost 模型（使用默认参数）
xgb_model = xgb.XGBClassifier(random_state=42)

# 训练模型
xgb_model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))

#### LightGBM

In [ ]:
# LightGBM
print('LightGBM')

# 定义 LightGBM 模型（使用默认参数）
lgb_model = lgb.LGBMClassifier(random_state=42)

# 训练模型
lgb_model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))
# y_pred_proba

### 调参，这里我们使用网格搜索（还可以使用optuna，启发式算法等）

#### 逻辑回归

In [ ]:
# 逻辑回归

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score


# 定义逻辑回归模型
lr = LogisticRegression()

# 定义参数网格
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 正则化强度的倒数
    'penalty': ['l1', 'l2'],       # 正则化类型
    'solver': ['liblinear']        # 优化算法（liblinear 支持 l1 和 l2）
}

# 初始化网格搜索
grid_search = GridSearchCV(
    estimator=lr,  # 模型
    param_grid=param_grid,  # 参数网格
    cv=5,  # 五折交叉验证
    scoring='roc_auc',  # 使用 AUC 作为评估指标
    n_jobs=-1  # 使用所有可用的CPU核心
)

# 在训练集上执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("最佳参数组合:", grid_search.best_params_)

# 使用最佳参数训练模型
best_lr = grid_search.best_estimator_  # best_lr 为最佳参数模型

# 在测试集上进行预测
y_pred_lr = best_lr.predict(X_test)

# # 输出混淆矩阵
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))

# # 输出分类报告
# print("Classification Report:\n", classification_report(y_test, y_pred_lr))

# 输出 AUC
y_pred_proba = best_lr.predict_proba(X_test)[:, 1]  # 获取正类的概率
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))

#### SVM

In [ ]:
# SVM
# 定义 SVM 模型
svm = SVC(probability=True, random_state=42)  # 启用概率估计

# 定义参数网格
param_grid = {
    'C': [0.1, 1, 10],  # 正则化参数
    'kernel': ['linear', 'rbf'],  # 核函数
    'gamma': ['scale', 'auto']  # 核函数的系数
}

# 初始化网格搜索
grid_search = GridSearchCV(
    estimator=svm,  # 模型
    param_grid=param_grid,  # 参数网格
    cv=5,  # 五折交叉验证
    scoring='roc_auc',  # 使用 AUC 作为评估指标
    n_jobs=-1  # 使用所有可用的CPU核心
)

# 在训练集上执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("最佳参数组合:", grid_search.best_params_)

# 使用最佳参数训练模型
best_svm = grid_search.best_estimator_

# 在测试集上进行预测
y_pred_svm = best_svm.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_svm))

#### 随机森林

In [ ]:
# 随机森林
# 定义随机森林模型
rf = RandomForestClassifier(random_state=42)

# 定义参数网格
param_grid = {
    'n_estimators': [50, 100, 200],  # 树的数量
    'max_depth': [None, 10, 20],     # 每棵树的最大深度
    'min_samples_split': [2, 5, 10], # 分裂内部节点所需的最小样本数
    'min_samples_leaf': [1, 2, 4],   # 叶子节点所需的最小样本数
    'max_features': ['sqrt', 'log2'] # 每棵树分裂时考虑的最大特征数
}

# 初始化网格搜索
grid_search = GridSearchCV(
    estimator=rf,  # 模型
    param_grid=param_grid,  # 参数网格
    cv=5,  # 五折交叉验证
    scoring='roc_auc',  # 使用 AUC 作为评估指标
    n_jobs=-1  # 使用所有可用的CPU核心
)

# 在训练集上执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("最佳参数组合:", grid_search.best_params_)

# 使用最佳参数训练模型
best_rf = grid_search.best_estimator_

# 在测试集上进行预测
y_pred_proba = best_rf.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))

#### XGBoost

In [ ]:
# XGBoost
# 定义 XGBoost 模型
xgb_model = xgb.XGBClassifier(random_state=42)

# 定义参数网格
param_grid = {
    'max_depth': [3, 6, 9],  # 树的最大深度
    'learning_rate': [0.01, 0.1, 0.3],  # 学习率
    'n_estimators': [100, 200, 300],  # 树的数量
    'subsample': [0.8, 1.0],  # 样本采样比例
    'colsample_bytree': [0.8, 1.0]  # 特征采样比例
}

# 初始化网格搜索
grid_search = GridSearchCV(
    estimator=xgb_model,  # 模型
    param_grid=param_grid,  # 参数网格
    cv=5,  # 五折交叉验证
    scoring='roc_auc',  # 使用 AUC 作为评估指标
    n_jobs=-1  # 使用所有可用的CPU核心
)

# 在训练集上执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("最佳参数组合:", grid_search.best_params_)

# 使用最佳参数训练模型
best_xgb = grid_search.best_estimator_

# 在测试集上进行预测
y_pred_proba = best_xgb.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))

#### LightGBM


In [ ]:
# LightGBM
# 定义 LightGBM 模型
lgb_model = lgb.LGBMClassifier(random_state=42)

# 定义参数网格
param_grid = {
    'num_leaves': [31, 63, 127],  # 每棵树的最大叶子节点数
    'learning_rate': [0.01, 0.1, 0.3],  # 学习率
    'n_estimators': [100, 200, 300],  # 树的数量
    'max_depth': [5, 10, -1],  # 树的最大深度，-1 表示不限制
    'subsample': [0.8, 1.0],  # 样本采样比例
    'colsample_bytree': [0.8, 1.0]  # 特征采样比例
}

# 初始化网格搜索
grid_search = GridSearchCV(
    estimator=lgb_model,  # 模型
    param_grid=param_grid,  # 参数网格
    cv=5,  # 五折交叉验证
    scoring='roc_auc',  # 使用 AUC 作为评估指标
    n_jobs=-1  # 使用所有可用的CPU核心
)

# 在训练集上执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("最佳参数组合:", grid_search.best_params_)

# 使用最佳参数训练模型
best_lgb = grid_search.best_estimator_

# 在测试集上进行预测
y_pred_proba = best_lgb.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 输出测试集 AUC
print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))

### 模型融合（这里直接使用调参后的模型，不考虑未条参的模型）

#### Stacking进行模型融合

**这里使用逻辑回归、随机森林、XGBoost作为基模型，LightGBM做元模型**

In [ ]:
# from sklearn.ensemble import StackingClassifier  # 导入 Stacking 分类器
# from sklearn.linear_model import LogisticRegression  # 逻辑回归模型
# from sklearn.ensemble import RandomForestClassifier  # 随机森林分类模型
# import xgboost as xgb  # XGBoost模型
# import lightgbm as lgb  # LightGBM模型
# # 定义基模型（使用调参后的最优参数）
# base_models = [
#     ('rf', RandomForestClassifier(
#         max_depth=20,  # 最优参数
#         max_features=0.8,  # 最优参数
#         min_samples_leaf=1,  # 最优参数
#         min_samples_split=2,  # 最优参数
#         n_estimators=120,  # 最优参数
#         random_state=42
#     )),  # 随机森林
#     ('xgb', XGBClassifier(
#         colsample_bytree=1.0,  # 最优参数
#         learning_rate=0.1,  # 最优参数
#         max_depth=9,  # 最优参数
#         n_estimators=150,  # 最优参数
#         reg_alpha=0,  # 最优参数
#         reg_lambda=1,  # 最优参数
#         subsample=0.8,  # 最优参数
#         random_state=42
#     )),  # XGBoost
#     ('lr', LogisticRegression(
#         colsample_bytree=1.0,  # 最优参数
#         learning_rate=0.1,  # 最优参数
#         num_leaves=100,  # 最优参数
#         subsample=0.8,  # 最优参数
#         random_state=42
#     ))  # 逻辑回归
# ]
# # 定义元模型（LightGBM）
# meta_model_lgbm = LGBMClassifier(
#     colsample_bytree=0.9,  # 微调
#     learning_rate=0.05,  # 降低学习率
#     num_leaves=50,  # 微调
#     n_estimators=200,  # 增加树的数量
#     reg_alpha=0.1,  # 微调 L1 正则化
#     reg_lambda=0.5,  # 微调 L2 正则化
#     subsample=0.9,  # 微调
#     random_state=42
# )

# # 创建 Stacking 回归器
# stacking_model_lgbm = StackingClassifier(
#     estimators=base_models,  # 基模型
#     final_estimator=meta_model_lgbm,  # LightGBM 作为元模型
#     n_jobs=1  # 使用1 CPU 核心
# )

# # 训练 Stacking 模型
# stacking_model_lgbm.fit(X_train, y_train)

# # 预测
# y_test_pred_lgbm = stacking_model_lgbm.predict(X_test)[:, 1]

# # 计算 AUC

# print("测试集 AUC: ", roc_auc_score(y_test, y_test_pred_lgbm))

#### 加权进行融合


In [ ]:
# # 定义基模型（使用你提供的参数）
# base_models = [
#     ('rf', RandomForestClassifier(
#         max_depth=20,  # 最优参数
#         max_features=0.8,  # 最优参数
#         min_samples_leaf=1,  # 最优参数
#         min_samples_split=2,  # 最优参数
#         n_estimators=120,  # 最优参数
#         random_state=42
#     )),  # 随机森林
#     ('xgb', XGBClassifier(
#         colsample_bytree=1.0,  # 最优参数
#         learning_rate=0.1,  # 最优参数
#         max_depth=9,  # 最优参数
#         n_estimators=150,  # 最优参数
#         reg_alpha=0,  # 最优参数
#         reg_lambda=1,  # 最优参数
#         subsample=0.8,  # 最优参数
#         random_state=42
#     )),  # XGBoost
#     ('lgbm', LGBMClassifier(
#         colsample_bytree=1.0,  # 最优参数
#         learning_rate=0.1,  # 最优参数
#         num_leaves=100,  # 最优参数
#         subsample=0.8,  # 最优参数
#         random_state=42
#     ))  # LightGBM
# ]

# # 训练基模型
# for name, model in base_models:
#     model.fit(X_train, y_train)
#     print(f"{name} 模型训练完成")

# # 获取每个模型的预测结果
# y_test_pred_rf = base_models[0][1].predict(X_test)  # 随机森林
# y_test_pred_xgb = base_models[1][1].predict(X_test)  # XGBoost
# y_test_pred_lgbm = base_models[2][1].predict(X_test)  # LightGBM

# # 定义网格搜索范围
# weights_rf = np.linspace(0, 1, 11)  # 随机森林权重范围 [0, 0.1, ..., 1]
# weights_xgb = np.linspace(0, 1, 11)  # XGBoost 权重范围 [0, 0.1, ..., 1]
# weights_lgbm = np.linspace(0, 1, 11)  # LightGBM 权重范围 [0, 0.1, ..., 1]

# # 初始化最佳 AUC 和最佳权重
# best_rmse = float('inf')
# best_weights = None

# # 网格搜索
# for w_rf in weights_rf:
#     for w_xgb in weights_xgb:
#         for w_lgbm in weights_lgbm:
#             if w_rf + w_xgb + w_lgbm == 1:  # 确保权重之和为 1
#                 # 加权平均
#                 y_test_pred_weighted = (w_rf * y_test_pred_rf +
#                                         w_xgb * y_test_pred_xgb +
#                                         w_lgbm * y_test_pred_lgbm)
#                 # 计算 RMSE
#                 test_rmse = mean_squared_error(y_test, y_test_pred_weighted, squared=False)
#                 # 更新最佳权重
#                 if test_rmse < best_rmse:
#                     best_rmse = test_rmse
#                     best_weights = (w_rf, w_xgb, w_lgbm)

# # 输出最佳权重和 RMSE
# print(f"最佳权重：随机森林={best_weights[0]:.2f}, XGBoost={best_weights[1]:.2f}, LightGBM={best_weights[2]:.2f}")
# print(f"最佳 AUC: {best_rmse:.4f}")

### 使用阈值进行二次调参 thresholds

In [ ]:
# from sklearn.metrics import accuracy_score, roc_auc_score

# model = stacking_model_lgbm

# # 在测试集上进行预测
# y_pred_proba = model.predict_proba(X_test)[:, 1]

# # 输出测试集 AUC
# print("测试集 AUC: ", roc_auc_score(y_test, y_pred_proba))

# # 初始化列表用于存储不同阈值下的准确率 --- 这里使用准确率作为评价指标
# accuracies = []
# thresholds = np.arange(0.1, 1, 0.01)

# for threshold in thresholds:
#     y_pred = (y_pred_proba >= threshold).astype(int)
#     accuracy = accuracy_score(y_test, y_pred)
#     accuracies.append(accuracy)

# # 找到最大准确率及其对应的阈值
# max_accuracy_index = np.argmax(accuracies)
# optimal_threshold = thresholds[max_accuracy_index]
# max_accuracy = accuracies[max_accuracy_index]

# print(f"最佳阈值: {optimal_threshold}")
# print(f"最大准确率: {max_accuracy}")

# # 绘制阈值 - 准确率曲线
# plt.plot(thresholds, accuracies)
# plt.xlabel('Threshold')
# plt.ylabel('Accuracy')
# plt.title('Threshold - Accuracy Curve')
# plt.show()

## 可解释分析

---

使用Shap库进行可视化分析

## 模型持久化

In [ ]:
# from joblib import dump,load

# # 保存模型
# dump(model, 'model.joblib')

# # 加载模型
# model = load('model.joblib')